**How to use Center Crop**  
The following functions are in IsoDAT2D.py, and this tutorial includes a potential main to run the functions in.

In [ ]:
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
import fabio
import SimDAT2D.SimDAT2D as sim

In [ ]:
def center_and_crop(image, crop_size=1400):
    """
    Displays an image, lets the user click 3 times to pick a center,
    then crops the image to a square region of `crop_size` x `crop_size`.
    """
    # Display image for clicks
    plt.figure("Select 3 Points", figsize=(14, 14))
    plt.imshow(image, vmin=0, vmax=1000000)
    plt.title("Click 3 times to select the center for cropping")
    plt.axis("on")
    
    # Capture exactly 3 clicks
    points = plt.ginput(3, timeout=-1)
    plt.close()
    
    if len(points) != 3:
        raise ValueError("You must click exactly 3 points.")
    
    # Compute the average click position
    center_x = int(np.mean([pt[0] for pt in points]))
    center_y = int(np.mean([pt[1] for pt in points]))
    print(f"Computed center: ({center_x}, {center_y})")
    
    # Calculate half-size
    half_size = crop_size // 2
    
    # Tentative boundaries
    x_min = center_x - half_size
    x_max = center_x + half_size
    y_min = center_y - half_size
    y_max = center_y + half_size
    
    # Ensure we don't go outside the image
    height, width = image.shape[:2]
    if x_min < 0:
        x_min = 0
        x_max = min(crop_size, width)
    if y_min < 0:
        y_min = 0
        y_max = min(crop_size, height)
    if x_max > width:
        x_max = width
        x_min = max(0, width - crop_size)
    if y_max > height:
        y_max = height
        y_min = max(0, height - crop_size)

    cropped_image = image[y_min:y_max, x_min:x_max]
    
    return cropped_image


In [ ]:
def create_line_mask(image):
    """
    Creates a mask where True indicates valid pixel
    """
    mask = (image > 0)
    return mask

In [ ]:
def preprocess_pilatus(file_name, crop_size=1400):
    """
    Loads a file, centers & crops (if needed), and creates a line mask.
    Returns (mask, corrected_image).
    """
    # Simple check to see if it's a CBF file
    if file_name.lower().endswith(".cbf"):
        print(f"Detected CBF file: {file_name}")
        image_data = fabio.open(file_name).data
        # Center & crop
        corrected_image = center_and_crop(image_data, crop_size=crop_size)
    else:
        print(f"Detected non-CBF file: {file_name}")
    
    # Create the mask to remove lines/gaps
    mask = create_line_mask(corrected_image)
    
    return mask, corrected_image


**Change file name and any dimensions you want to alter depending on the size of your tiff or cbf file**

In [ ]:
if __name__ == "__main__":
    file_name = "da_7_c_gst_sio2.cbf"  # Replace with your file path
    #Preprocess the image (center, crop, and create a line mask)
    mask, cropped_img = preprocess_pilatus(file_name, crop_size=1400)
    
    #Use the mask to set invalid pixels to NaN in the cropped image.
    processed_img = np.where(mask, cropped_img, np.nan)
    
    #Define the circular region parameters.
    circle_center = (processed_img.shape[0] // 2, processed_img.shape[1] // 2)
    circle_radius = 700
    circled_img = fill_with_circle(processed_img, radius=circle_radius, center=circle_center)
    
    #Fill in the NaN values within the circle using your second function.
    filled_img = fill_nan_with_neighbor_mean_circle(circled_img, radius=circle_radius, center=circle_center)
    
    # Display the final processed image
    plt.figure("Final Processed Image", figsize=(12, 12))
    plt.imshow(circled_img, cmap='magma', vmin=0, vmax=1000000)
    plt.title("Cropped Image with Circular Mask and Filled NaNs")
    plt.axis("off")
    plt.show()